# Image Compressor and Streamer

### Source
#### Compress images using JPEG lossy compression, serialize and send via socket

### Destination
#### Receive compressed image via listener and deserialize and decompress to reconstruct image

In [1]:
# load libraries
import cv2
import sys
import numpy as np
import pickle as pickle
import struct
import socket
import zstd # compression library from Facebook
from IPython.display import HTML

In [ ]:
# Server side:
def recvall(sock, count):
    buf = b''
    while count:
        newbuf = sock.recv(count)
        if not newbuf:
        	print("unable to get stream") 
        	return None
        buf += newbuf
        count -= len(newbuf)
    return buf

HOST=''
PORT=8090 # port 8090

s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
print('Socket created')

s.bind((HOST,PORT))
print('Socket bind complete')
s.listen(True)
print('Socket now listening')

conn,addr=s.accept()

while True:
    length = recvall(conn,16)
    print("size of bytestream received: " + str(length))
    print("deserializing: ")
    frame_compressed = pickle.loads(recvall(conn, int(length)))
    print(sys.getsizeof(frame_compressed))
    
    # decompress frame
    string_data = zstd.decompress(frame_compressed)
    print("size of numpy string after decompression: " + str(sys.getsizeof(string_data)))
    
    # get numpy array and decode the image
    data = np.fromstring(string_data, dtype='uint8')
    decimg=cv2.imdecode(data,1)
    print("decoded image size: " + str(sys.getsizeof(decimg)))
    
    cv2.imshow("server_side",decimg)
    print(" ")
    if cv2.waitKey(25) & 0xFF==ord('q'):
        break
cv2.destroyAllWindows()
cap.release()
s.close()

Socket created
Socket bind complete
Socket now listening


In [ ]:
# Client (source):
# socket connection establishment
clientsocket=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
clientsocket.connect(('localhost',8090)) # port to connect to server

# stream using opencv and store video file
cap=cv2.VideoCapture(0) #webcam

# jpeg lossy compression
encode_param=[int(cv2.IMWRITE_JPEG_QUALITY),50]

while (True):
    ret,image_np_org =cap.read()
    image_np = image_np_org
    #image_np = cv2.resize(image_np_org, (416,416))
    print("original image size: " + str(sys.getsizeof(image_np)))
    cv2.imshow("original",image_np)
    
    print("encoding - lossy jpeg compression:")
    result, imgencode = cv2.imencode('.jpg', image_np, encode_param)
    print("size of image after encoding: " + str(sys.getsizeof(imgencode)))
    
    print("converting to numpy array:")
    data_img = np.array(imgencode)
    stringData = data_img.tostring()
    print("after encoding and converting to numpy array: " + str(sys.getsizeof(stringData)))
    
    # zstd compression
    data_compressed = zstd.compress(stringData,20)
    print("after compression: " + str(sys.getsizeof(data_compressed)))
    
    data = pickle.dumps(data_compressed)
    print("after serialization: " + str(sys.getsizeof(data)))
    print("sending via socket: ")
    clientsocket.send((str(len(data)).ljust(16)).encode('utf-8','ignore'))
    clientsocket.send(data)
    print(" ")
    if cv2.waitKey(25) & 0xFF==ord('q'):
        break
cv2.destroyAllWindows()
cap.release()
clientsocket.close()

### Compressing 920KB frame to 23KB and back..

In [13]:
HTML('<img src="image_compressor.gif" width="100%">')

In [12]:
# image snapshot
HTML('<img src="image_compressor_snapshot.png" width="100%">')